## pdfの論文を全部OCRで文字起こしするためのコード
## （表などあると汚くなるので微妙...）

In [1]:
import re

from google.cloud import storage
from google.cloud import vision_v1p2beta1 as vision
from google.protobuf import json_format

from time import sleep
import sys

In [2]:
def async_detect_document(gcs_source_uri, gcs_destination_uri):
    # Supported mime_types are: 'application/pdf' and 'image/tiff'
    mime_type = 'application/pdf'

    # How many pages should be grouped into each json output file.
    # With a file of 5 pages
    #batch_size = 2
    batch_size = 5

    client = vision.ImageAnnotatorClient()

    feature = vision.types.Feature(
        type=vision.enums.Feature.Type.DOCUMENT_TEXT_DETECTION)

    gcs_source = vision.types.GcsSource(uri=gcs_source_uri)
    input_config = vision.types.InputConfig(
        gcs_source=gcs_source, mime_type=mime_type)

    gcs_destination = vision.types.GcsDestination(uri=gcs_destination_uri)
    output_config = vision.types.OutputConfig(
        gcs_destination=gcs_destination, batch_size=batch_size)

    async_request = vision.types.AsyncAnnotateFileRequest(
        features=[feature], input_config=input_config,
        output_config=output_config)

    operation = client.async_batch_annotate_files(
        requests=[async_request])

    print("Operation started: {}".format(operation.operation))

    return operation

In [21]:
gcs_source_uri = "gs://work_kanouchi/ocr/in/A1-1.pdf"
gcs_destination_uri = "gs://work_kanouchi/ocr/out/A1-1"
req= async_detect_document(gcs_source_uri, gcs_destination_uri)

/Users/kanouchishin/py3.7/lib/python3.7/site-packages/google/auth/_default.py:66: UserWarning: Your application has authenticated using end user credentials from Google Cloud SDK. We recommend that most server applications use service accounts instead. If your application continues to use end user credentials from Cloud SDK, you might receive a "quota exceeded" or "API not enabled" error. For more information about service accounts, see https://cloud.google.com/docs/authentication/
  warnings.warn(_CLOUD_SDK_CREDENTIALS_WARNING)


Operation started: name: "projects/usable-auth-library/operations/30da48c8eccc9bc0"



In [22]:
print(req)

In [8]:
wait_time = 15
sleep(wait_time)

In [23]:
while (True):
    if req.done():
        break

    print("Operation not completed. Stil waiting...")
    sleep(wait_time)
    wait_time += 10

In [24]:
storage_client = storage.Client()
blob_list = []

/Users/kanouchishin/py3.7/lib/python3.7/site-packages/google/auth/_default.py:66: UserWarning: Your application has authenticated using end user credentials from Google Cloud SDK. We recommend that most server applications use service accounts instead. If your application continues to use end user credentials from Cloud SDK, you might receive a "quota exceeded" or "API not enabled" error. For more information about service accounts, see https://cloud.google.com/docs/authentication/
  warnings.warn(_CLOUD_SDK_CREDENTIALS_WARNING)
/Users/kanouchishin/py3.7/lib/python3.7/site-packages/google/auth/_default.py:66: UserWarning: Your application has authenticated using end user credentials from Google Cloud SDK. We recommend that most server applications use service accounts instead. If your application continues to use end user credentials from Cloud SDK, you might receive a "quota exceeded" or "API not enabled" error. For more information about service accounts, see https://cloud.google.com

In [25]:
match = re.match(r'gs://([^/]+)/(.+)', gcs_destination_uri)

if match :

    bucket_name = match.group(1)
    prefix = match.group(2)

    bucket = storage_client.get_bucket(bucket_name=bucket_name)

    # List objects with the given prefix.
    blob_list = list(bucket.list_blobs(prefix=prefix))
    
else:
    print("Pattern not matched!")

In [26]:
print('Output files:')


for blob in blob_list:
    print(blob.name)

Output files:
ocr/out/A1-1output-1-to-4.json


In [ ]:
output = blob_list[0]

json_string = output.download_as_string()

response = json_format.Parse(
    json_string, vision.types.AnnotateFileResponse())

# The actual response for the first page of the input file.
first_page_response = response.responses[0]
annotation = first_page_response.full_text_annotation


In [ ]:
print(u'Full text:\n{}'.format(
    annotation.text))

In [15]:
import json

filename = "/Users/kanouchishin/Downloads/ocr_out_output-1-to-4.json"
fp = open(filename)
data = json.load(fp)
data['responses'][0]['fullTextAnnotation']['text']

"言語処理学会 第25回年次大会 発表論文集(2019年3月)\n日本語から英語への文脈翻訳テストの提案\n永田 昌明 森下睦\nNTT コミュニケーション科学基礎研究所\n{nagata.masaaki, morishita.makoto}@lab.ntt.co.jp\n1 はじめに\nとなる連続する2つの英語文を作成した。さらに、正解\nと対照可能な不正解として、英語の第2文に対して文\n脈を考慮しないことにより生じる必要最小限の誤りを\n加えた英語文を作成した。この際、できるだけ正解率\nが50%になるよう工夫した。ここで考慮する文脈情報\nは共参照 (coreference) と一貫性 (coherence/cohesion)\nとし、その総数は約 1000件である。\n2.1 共参照\nニューラル機械翻訳の登場により文単位の翻訳精度\nは大きく向上し、さらに翻訳精度を上げるために文脈\nを利用する研究が始まった [15, 1, 17] 。文脈を利用す\nる最も簡単な方法は、原言語および目的言語において\n直前の文と現在の文を <concat> のような特別な区切\nり記号で連結し、通常の文単位の翻訳を行うことでで\nある [15]。この方法は 2-to-2 と呼ばれ、最先端の文脈\n翻訳手法と大差ない精度が得られることが知られてい\nる [1, 17]。これに対して通常の文単位の翻訳は 1-to-1\nと呼ばれる。また入力文と文脈に別々の encoder を使\n用する方法も提案されており、encoder として RNN[1]\nや transformer[17] が使われている。\n文脈の利用法を関する研究の問題点の一つは、BLEU\nのような従来の自動評価尺度では、文脈の何が問題で\nそれが提案手法によりどう改善されたのかよくわか\nらないことである。文献 [1] では、対照テストセット\n(contrastive test set)[11] の枠組みに基づいて、英語\nからフランス語への翻訳において機械翻訳が文脈を理\n解して適切な訳文を生成する能力を評価する文脈テス\nトセットを提案した\n本稿では日本語から英語への翻訳の文脈テストの作\n成法を提案する。我々は当初、文献 [1] の方法に基づ\nいて共参照と一貫性に関する文脈テストを作成しよう\n

In [16]:
data['responses'][1]['fullTextAnnotation']['text']

"We aren't allowed to use a (*the) mobile\nphone in class.\n照応関係があり、一方がaで他方が the であるような\n英語文のデータ OnteNotes の coreference chain から\n容易に見つけることができる。\n2.2 一貫性\n昨日、渋谷へ行った。\nすごい人だった。\nSource:\nContext:\nInput:\nTarget:\nContext:\nCorrect:\nIncorrect:\nI went to Shibuya yesterday.\nThere are a lot of people.\nHe is a great man.\n(a) 曖昧性解消\nいい時計ですね。\nこの時計は父の形見なんです。\nSource:\nContext:\nInput:\nTarget:\nContext:\nCorrect:\nIncorrect:\nIt's a nice clock.\nThis clock is a memento of my father.\nThis watch is a memento of my father.\nされる。それに対して英語は構文上の制約から主語や\n目的語が必須である。そのため英語側の代名詞の人称、\n数、格を正しく決定しなければならない。図 1(a) の\n例では、文脈がなければ「思う」のような思考動詞や\n「診察する」のような意思を伴う行為動詞のデフォー\nルトの主語は一人称であるが、ここでは前の文に登場\nする人物を指していることから正解は三人称になる。\n文献 [1] では、OpenSubtities にある実例を参考に\nして文脈テストを作成している。当初、我々も Open-\nSubtitles や Ted Talk のコーパスからテストを作成し\nようとしたが、日本語のゼロ代名詞のデフォールト訳\nが一人称または二人称であることが多いという視点が\n欠け、不自然になることが多かった。また日本人の作\n業者でも、ゼロ代名詞を正しく分析するのは難しく、\n作業誤りが目立った。\nそこでまず信頼できるゼロ代名詞の分析結果とし\nて Keyaki Treebank[2] を用いることにした。具体的\nには、まず日本語のゼ

In [18]:
data['responses'][2]['fullTextAnnotation']['text']

'図 2(b) に対応に関するテストの例を示す。時計の\n英訳は clock または watch であるが、同一文脈で同一\n対象を指示する際に2つの訳が混在してはいけない。\n一般に一貫性に関するテストは以下の3つの条件を満\nたす。\nDataset\nIWSLT2017\n(TED Talks)\nlen(ja)\n22.3\n21.8\n22.5\n7.6\n9.0\n・ 原言語文は多義を持つ語句を含む。\nOpenSubtitles2018\n(movie subtitles)\n6.9\n・ その多義は目的言語の異なる語句に翻訳される。\n目的言語ではこれらはほとんど別概念なので、相\n互に入れ替えることは不可能である。\nGlobal Voices\n(blog)\ntest\n・ 原言語の先行文でも同じ語句単語を含み、当該語\n句の意味の違いに応じて、目的言語では現在の文\nと同じ多義が生じる。\nlen(en)\n20.6\n19.1\n19.5\n8.5\n7.7\n8.9\n21.6\n22.0\n21.8\n22.0\n20.3\n21.3\n21.1\n19.8\n19.9\n14.9\n15.1\n13.1\n24.9\nHiragana Times\nBooks\n(book)\nHiraganaTimes\n(magazine)\nSplit sents\ntrain 218,174\ndev 2,577\ntest 2,357\ntrain 2,077,430\ndev 3,245\ntest\n2,901\ntrain 29,508\ndev\n9,426\n8,148\ntrain 16,472\ndev 2,792\ntest\ntrain 189,925\ndev 5,385\ntest 5,004\ntrain 103,417\ndev 4,279\n3,212\ntrain 480,778\n1,257\ntest\n1,287\ntrain 283,710\ndev\n3,002\n3,014\ntrain | 3,399,414\ndev 31,963\ntest | | 28,460\n2,537\n目的言語ではほぼ別概念なので、先行文と同じ語句を\n使う必要があるという制約は、言語対や翻訳方向によ\nらない一般的

In [19]:
data['responses'][3]['fullTextAnnotation']['text']

'3.2 - 翻訳精度と文脈翻訳テスト\n参考文献\n[1] Rachel Bawden, Rico Sennrich, , Alexandra Birch, and Barry\nHaddow. Evaluating Discourse Phenomena in Neural Ma-\nchine Translation. In Proceedings of the NAACL-2018, pp.\n1304-1313, 2018.\n訓練データから 1-to-1 と 2-to-2 のモデルを作成し、\n各データのテストセットで翻訳精度 (BLEU)を測定し\nた結果を表2に示す。 2-to-2 の翻訳精度は 1-to-1 に比\nべて一貫して良いがその差は小さい。\nOpenNMT-lua の -tgt オプションを使って、正解文\n対と不正解文対を強制翻訳 (forced decoding) し、正\n解文対の対数確率が不正解文対より高いテストの割合\nを文脈翻訳テストの正解率とする。表3に各テストに\n対する 1-to-1 と 2-to-2 の正解率を示す。\n[2] Alastair Butler, Tomoko Hotta, Ruriko Otomo, Kei Yoshi-\nmoto, Zhen Zhou, and Hong Zhu. Keyaki treebank: Phrase\nstructure with functional information for japanese. In Pro-\nceedings of Text Annotation Workshop, 2012.\n[3] Mauro Cettolo, Marcello Federico, Luisa Bentivogli, Jan\nNiehues, Sebastian Stüker, Katsuhito Sudoh, Koichiro\nYoshino, and Christian Federmann. Overview of the iwslt\n2017 evaluation campaign. In Proceedings of the IWSLT-\n2017, pp. 2-14, 2017.\n[4] Robert Dale, Ilya Anisimoff, a